In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [175]:
df = pd.read_csv("data/new_sg_companies_reviews.csv")

In [176]:
df.isna().sum()

Company Name         0
Overall Rating       0
Review Date          0
Review Title       109
Job Title            8
Job Details          0
Location          5772
Pros                 0
Cons                 0
dtype: int64

In [177]:
df.shape

(20035, 9)

In [178]:
def format_date(x, input_format = "%d-%b-%y", ignore_errors = False):
    try:
        return datetime.strptime(x, input_format)
    except Exception as e:
        if ignore_errors:
            return x
        else:
            raise e
    
def format_non_conventional_dates(x):
    if isinstance(x, datetime):
        return x
    if not isinstance(x, float):
        try:
            x = float(x)
        except:
            print(f"{x} unknown date type")
            return x
    return pd.Timedelta(x, unit='d') + datetime(1899, 12, 30)

def clean_df_dates(df, date_col, **args):
    df[date_col] = df[date_col].apply(lambda x: format_date(x, input_format = "%d-%b-%y", **args))
    df[date_col] = df[date_col].apply(lambda x: format_date(x, input_format = "%b %d, %Y", **args))
    df[date_col] = df[date_col].apply(lambda x: format_non_conventional_dates(x))
    return df

df = clean_df_dates(df, "Review Date", ignore_errors = True)

In [179]:
df = df.reset_index(names="id")
df['id'] = df.apply(lambda x: f"{x['Review Date']}{x['Review Title']}{x['id']}", axis = 1)
df.to_csv("data/new_sg_companies_reviews_UID.csv", index = False)

In [180]:
def clean_yj_style(df):
    df.replace('#NAME?', pd.NA, inplace=True) # replace '#NAME?' with NaN
    # Replace '' with NaN
    df.replace('', pd.NA, inplace=True) # replace '' with NaN
    # df.replace('•', '', inplace=True) # replace '•' with ''
    # df.replace('’', "'", inplace=True) # replace '’' with "'"
    df.replace('\u2022', '', regex=True, inplace=True) # replace '•' with ''
    df.replace('\u2019', "'", regex=True, inplace=True) # replace '’' with "'"
    df.replace('\r\n\-', ' ', regex=True, inplace=True) # replace '\r\n-' with ' '
    df.replace('\r\n\*', ' ', regex=True, inplace=True) # replace '\r\n*' with ' '
    df.replace('\r\n•', ' ', regex=True, inplace=True) # replace '\r\n•' with ' '
    df.replace('\n\r\-', ' ', regex=True, inplace=True) # replace '\n\r-' with ' '
    df.replace('\n\r\*', ' ', regex=True, inplace=True) # replace '\n\r*' with ' '
    df.replace('\n\r•', ' ', regex=True, inplace=True) # replace '\n\r•' with ' '
    df.replace('\n\-', ' ', regex=True, inplace=True) # replace '\n-' with ' '
    df.replace('\n\*', '', regex=True, inplace=True) # replace '\n*' with ' '
    df.replace('\n•', ' ', inplace=True) # replace '\n•' with ' '
    df.replace('\r-', ' ', inplace=True) # replace '\r-' with ' '
    df.replace('\r*', ' ', inplace=True) # replace '\r*' with ' '
    df.replace('\r•', ' ', inplace=True) # replace '\r•' with ' '
    df.replace('\n',' ', regex=True, inplace=True) # replace '\n' with ' '
    df.replace('\r',' ', regex=True, inplace=True) # replace '\r' with ' '
    df.replace('\t',' ', regex=True, inplace=True) # replace '\t' with ' '
    df['Pros'] = df['Pros'].map(lambda x: x.lstrip('- ') if isinstance(x, str) else x)
    df['Cons'] = df['Cons'].map(lambda x: x.lstrip('- ') if isinstance(x, str) else x)
    df['Pros'] = df['Pros'].map(lambda x: x.lstrip('* ') if isinstance(x, str) else x)
    df['Cons'] = df['Cons'].map(lambda x: x.lstrip('* ') if isinstance(x, str) else x)
    return df

In [181]:
from cleantext import clean
import emoji
import re

def convert_emoji_to_text(emoji_text):
    try:
        text_with_aliases = emoji.demojize(emoji_text, delimiters=("", "_emoji "))
    except:
        return emoji_text
    return text_with_aliases

def is_ascii(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
        return True
    except UnicodeDecodeError:
        return False

def get_non_ascii_strings(df, col):
    # Use apply function to check each value in the specified column
    non_ascii_mask = df[col].apply(lambda x: not is_ascii(str(x)))

    # Return a DataFrame containing non-ASCII strings
    return df[non_ascii_mask]

def clean_text(df, col):
    df = df.copy()
    df[col] = df[col].apply(lambda x: clean(x, no_emoji=False, no_punct=True) if isinstance(x, str) else x)
    df[col] = df[col].apply(lambda x: convert_emoji_to_text(x))
    df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
    df[col] = df[col].apply(lambda x: re.sub(r' {2,}', ' ', x) if isinstance(x, str) else x)
    return df

index_column = ['id']
df_index = df[index_column]
df = df.drop(columns=index_column)

text_columns = ['Review Title', 'Job Title', 'Job Details', 'Pros', 'Cons']
df_clean = clean_yj_style(df)
for column_to_check in text_columns:
    df_clean = clean_text(df_clean, column_to_check)
    result = get_non_ascii_strings(df_clean, column_to_check)
    print(f"Non-ASCII strings in '{column_to_check}':")
    print(len(result))
    if len(result) > 0:
        display(result)

df_clean = pd.concat([df_index, df_clean], axis = 1)


Non-ASCII strings in 'Review Title':
0
Non-ASCII strings in 'Job Title':
0
Non-ASCII strings in 'Job Details':
0
Non-ASCII strings in 'Pros':
0
Non-ASCII strings in 'Cons':
0


In [182]:
print(df.iloc[0]['Review Title'])
print("==========================")
print("After cleaning")
print("==========================")
print(df_clean.iloc[0]['Review Title'])

Good pay, hard work
After cleaning
good pay hard work


In [183]:
df_clean.dtypes

id                        object
Company Name              object
Overall Rating           float64
Review Date       datetime64[ns]
Review Title              object
Job Title                 object
Job Details               object
Location                  object
Pros                      object
Cons                      object
dtype: object

In [184]:
df_clean = df_clean.drop_duplicates(subset = [i for i in df_clean.columns if i != 'id']).reset_index(drop = True)
df_clean.to_csv("data/new_sg_companies_reviews_clean_UID.csv", index = False)

In [185]:
# df_clean.to_csv("data/new_sg_companies_reviews_clean.csv", index = False)

## Indexing classes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

In [2]:
df = pd.read_csv("data/final_sg_companies_reviews_clean.csv")

In [3]:
df_uid = pd.read_csv("data/new_sg_companies_reviews_clean_UID.csv")
df_uid = df_uid.dropna(subset=['Review Title','Pros', 'Cons']).reset_index(drop = True)

In [4]:
assert df.equals(df_uid.drop(columns=['id']))
df_uid.to_csv("data/final_sg_companies_reviews_clean_UID.csv", index=False)

In [192]:
df = pd.read_csv("data/final_sg_companies_reviews_clean_UID.csv")

In [193]:
from retrievers import BaseRetriever
from enums import EncoderType, IndexerType
encoder_type = EncoderType("TermFrequencyEncoder")
# encoder_type = EncoderType("SentenceTransformerEncoder")
indexer_type = IndexerType("LeaderIndexer")
corpus = df['Review Title'].to_numpy()

start_indexing = time.time()
retriever = BaseRetriever(encoder_type, indexer_type, corpus, encoder_kwargs={}, indexer_kwargs={"n_clusters": 1, 'use_pca': False, 'min_explained_var': 0.7})
end_indexing = time.time()
print(f"time taken for indexing: {end_indexing - start_indexing}s")

time taken for indexing: 0.27309465408325195s


In [194]:
q1 = "I want to see good MONEY, CULTURE AND WORK LIFE BALANCE"
q2 = "career progression???"
q3 = "the worst companies to work at"
q4 = "tiktok software engineer"

start_retrieving = time.time()
results = retriever.retrieve_results(q4)
end_retrieving = time.time()
print(f"time taken for retrieval: {end_retrieving - start_retrieving}s")

time taken for retrieval: 0.00640869140625s


In [195]:
df.iloc[results]

,id,Company Name,Overall Rating,Review Date,Review Title,Job Title,Job Details,Location,Pros,Cons
2108,2024-02-08 00:00:00TikTok2592,TikTok,4.0,2024-02-08,tiktok,marketing operations,current employee,"New York, NY",its fun and great to work there,its chaotic sometimes but good
2128,2024-02-06 00:00:00TikTok Review2618,TikTok,3.0,2024-02-06,tiktok review,agency development manager,former employee,"Austin, TX",very fun place to work,revenue has gone down so they did layoffs
5206,2017-05-15 00:00:00Software Engineer6266,ST Electronics (Singapore),4.0,2017-05-15,software engineer,anonymous employee,former employee,NaN,a good culture and environment to work with,no team work and kinda stressful
11205,2023-11-10 00:00:00Software Engineer12814,PCCW,4.0,2023-11-10,software engineer,software engineer,former employee more than 1 year,"Manila, Manila",a lot of good projects to learn from good sala...,project timelines are too demanding sometimes
4825,2023-11-06 00:00:00Software engineer5830,X,4.0,2023-11-06,software engineer,software engineer,former employee more than 1 year,"San Francisco, CA",good company to work and learn,nothing much all was fine until it was acquired
...,...,...,...,...,...,...,...,...,...,...
12199,2023-01-31 00:00:00Not DEI friendly; poor econ...,Groupon,1.0,2023-01-31,not dei friendly poor economic outlook they ar...,anonymous employee,current employee,NaN,there are not many positives besides some comm...,groupon is not dei friendly i have various ema...
12198,"2023-01-12 00:00:00Fun, Collaborative, Thought...",Groupon,5.0,2023-01-12,fun collaborative thought provoking environment,manager,current employee more than 3 years,NaN,unlimited paid time off diverse culture inclus...,change management but optimistic in finding ne...
12197,2023-08-08 00:00:00Great place to work13815,Groupon,3.0,2023-08-08,great place to work,data scientist,current employee,"Chicago, IL",great place to work and learn,not performing well due to recession
12196,"2023-05-22 00:00:00Great place to learn, but h...",Groupon,3.0,2023-05-22,great place to learn but has no soul,anonymous employee,former employee more than 3 years,"Seattle, WA",good wlb competitive pay friendly team 100 rem...,i got laid off leadership cant decide what to ...
